# Risk Data Generator

The purpose of this script is to convert simulated data (raw output) into risk scores and store these for each class of simulations each with their own parameter regimes. "Run all cells" will create a set of risk matrices stored in npz form in the './data/' folder. These matrices are then access for performance reporting. 

In [1]:
import numpy as np
import sys
import matplotlib
sys.path.append('../')

from visualiserisk import *
from qslamdesignparams import GRIDDICT

#### Parameter Regimes

The set of parameters are given in the dictionary format, LOOPS_DICT, and used for analysis via ParamUpdater. 

In [2]:
meta_truth_floor_scan = [0.2, 0.4, 0.6, 0.8, 1.0]

meta_max_iter_scan = [ 5, 10, 15, 20, 25, 50, 75, 100, 125, 250]

noiseclasses = ['noiseless'] + ['alwaysdark', 'spnoise']*4
noisestrengths = [0.0, 0.1, 0.1, 0.2, 0.2, 0.3, 0.3, 0.5, 0.5]*2 # shouldnt be a 2 but hopefully doesnt matter
meta_noisevar_scan = zip(noiseclasses, noisestrengths)

lowscan = np.asarray([0.2]*5)*np.pi
highscan = np.asarray([0.2, 0.4, 0.6, 0.8, 1.0])*np.pi
truth_step_scan = zip(lowscan, highscan)


lambda1 = [0.99, 0.956, 0.922, 0.888, 0.854, 0.820, 0.786, 0.752, 0.718, 0.684, 0.65]
lambda2 = [0.977, 0.9752, 0.9734, 0.9716, 0.9698, 0.968, 0.9662, 0.9644, 0.9626, 0.9608, 0.959]
lambda_scan = zip(lambda1, lambda2)
msmt_per_qubit_scan = [1, 2, 4, 5, 6, 8, 10, 15, 20, 25, 50]

msmt_per_qubit_scan = [1, 2, 4, 5, 6, 8, 10, 15, 20, 25, 50]


LOOPS_DICT = {"meta_truth_floor_scan": meta_truth_floor_scan,
              "meta_max_iter_scan":meta_max_iter_scan, 
              "meta_noisevar_scan": meta_noisevar_scan,
              "truth_step_scan": truth_step_scan,
              "lambda_scan":lambda_scan,
              "msmt_per_qubit_scan": msmt_per_qubit_scan}

ParamUpdater = DataCube(LOOPS_DICT)


Where to find the './data/' folder:

In [3]:
loadpath = ''

#### Risk score generator script:

In [4]:
# Data variables
max_msmtvar = len(ParamUpdater.meta_max_iter_scan)
max_var = max(len(ParamUpdater.lambda_scan), len(ParamUpdater.msmt_per_qubit_scan))


for prefix in ['NSL_tfloor', 'NSL_theight']:

    if prefix[-5:] == 'floor':
        max_prevar_floor = len(ParamUpdater.meta_truth_floor_scan)
    if prefix[-5:] == 'eight':
        max_prevar_floor = len(ParamUpdater.truth_step_scan)

    for idx_noise_var in range(len(ParamUpdater.meta_noisevar_scan)):

        for idx_var_dict in range(2):

            full_data_matrix = np.zeros((max_prevar_floor, max_msmtvar, max_var, 2, 4))

            for idx_prevar in range(max_prevar_floor):

                for idx_msmt_var in range(max_msmtvar):

                    macro_performance = []

                    for idx_var in range(max_var):

                        regime_ID = prefix + '_n_' + str(idx_noise_var) +'_vset_' + str(idx_var_dict)
                        testcase_ID = regime_ID + '_t_' + str(idx_prevar) + '_m_' + str(idx_msmt_var)

                        unique_ID = loadpath + testcase_ID + '_v_' + str(idx_var)

                        PATHDICT["fle"] = unique_ID

                        perf_metrics = np.zeros((2, 4))
                        
                        try:
                            qslamdata = np.load(path_to_file(PATHDICT, flag='q'))
                            naivedata = np.load(path_to_file(PATHDICT, flag='n'))
                        except:
                            print "Data filenot loaded. Unique ID:", unique_ID
                            macro_performance.append(perf_metrics)
                            continue

                        perf_metrics[:, 0] = np.asarray([Metric.original_err_metric(data["macro_residuals"]) for data in [qslamdata, naivedata]])
                        perf_metrics[:, 1] = np.asarray([Metric.err(data["macro_residuals"]) for data in [qslamdata, naivedata]])
                        perf_metrics[:, 2] = np.asarray([Metric.rms(data["macro_residuals"]) for data in [qslamdata, naivedata]])
                        perf_metrics[:, 3] = np.asarray([Metric.ssim(data)[0] for data in [qslamdata, naivedata]]) # returning ssim (raw) or deviations

                        macro_performance.append(perf_metrics)


                    full_data_matrix[idx_prevar, idx_msmt_var, :, :, :] = np.asarray(macro_performance)


            datafile = './data/'+'RISK_' + regime_ID +'.npz'
            np.savez(datafile, full_data_matrix=full_data_matrix)
            
            print "Made data file:", datafile


Made data file: ./data/RISK_NSL_tfloor_n_0_vset_0.npz
Made data file: ./data/RISK_NSL_tfloor_n_0_vset_1.npz
Made data file: ./data/RISK_NSL_tfloor_n_1_vset_0.npz
Made data file: ./data/RISK_NSL_tfloor_n_1_vset_1.npz
Made data file: ./data/RISK_NSL_tfloor_n_2_vset_0.npz
Made data file: ./data/RISK_NSL_tfloor_n_2_vset_1.npz
Made data file: ./data/RISK_NSL_tfloor_n_3_vset_0.npz
Made data file: ./data/RISK_NSL_tfloor_n_3_vset_1.npz
Made data file: ./data/RISK_NSL_tfloor_n_4_vset_0.npz
Made data file: ./data/RISK_NSL_tfloor_n_4_vset_1.npz
Made data file: ./data/RISK_NSL_tfloor_n_5_vset_0.npz
Made data file: ./data/RISK_NSL_tfloor_n_5_vset_1.npz
Made data file: ./data/RISK_NSL_tfloor_n_6_vset_0.npz
Made data file: ./data/RISK_NSL_tfloor_n_6_vset_1.npz
Made data file: ./data/RISK_NSL_tfloor_n_7_vset_0.npz
Made data file: ./data/RISK_NSL_tfloor_n_7_vset_1.npz
Made data file: ./data/RISK_NSL_tfloor_n_8_vset_0.npz
Made data file: ./data/RISK_NSL_tfloor_n_8_vset_1.npz
Made data file: ./data/RISK_